In [1]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/2024-05-04-16-05-53.zip
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 19.5M/19.5M [00:00<00:00, 62.2MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/ir-lab-sose-2024/ir-acl-anthology-20240504-training/tira-ir-starter


In [4]:
#Erstellen von costum Retrieval Systemen

def create_index(documents, stopwords):
    # Stemmer has been added 
    # indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='EnglishSnowballStemmer', stopwords=stopwords)
    # indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='PorterStemmer', stopwords=stopwords)
    # indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='WeakPorterStemmer', stopwords=stopwords)
    # indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='StemmerTermPipeline', stopwords=stopwords)
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='LemurKrovetzStemmer', stopwords=stopwords)
    
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

def read_text_file_to_array(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Remove newline characters and convert to integers
            array = [(line.strip()) for line in lines]
            return array
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

# Read txt file and convert it to an array
file_path = "../terrier-stopwordslist.txt"

stopwords = read_text_file_to_array(file_path)


costum_index = create_index(pt_dataset.get_corpus_iter(), stopwords)

XSqrA_M	 = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_costum	 = pt.BatchRetrieve(costum_index, wmodel="XSqrA_M")

File ../terrier-stopwordslist.txt not found.


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 0/126958 [00:00<?, ?it/s]

09:36:10.345 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - TermPipeline object org.terrier.terms.LemurKrovetzStemmer not found: java.lang.ClassNotFoundException: org.terrier.terms.LemurKrovetzStemmer


java.lang.ClassNotFoundException: org.terrier.terms.LemurKrovetzStemmer
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:581)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.terrier.utility.ApplicationSetup.getClass(ApplicationSetup.java:416)
	at org.terrier.structures.indexing.Indexer.load_pipeline(Indexer.java:323)
	at org.terrier.structures.indexing.Indexer.init(Indexer.java:197)
	at org.terrier.structures.indexing.classical.BasicIndexer.<init>(BasicIndexer.java:183)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingCo

In [5]:
#Evaluation

pt.Experiment(
    [XSqrA_M, XSqrA_M_costum],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names = ["XSqrA_M", "XSqrA_M_costum"]
)

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
09:36:43.175 [main] ERROR org.terrier.terms.BaseTermPipelineAccessor - TermPipeline object org.terrier.terms.LemurKrovetzStemmer not found
java.lang.ClassNotFoundException: org.terrier.terms.LemurKrovetzStemmer
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:581)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.terrier.utility.ApplicationSetup.getClass(ApplicationSetup.java:421)
	at org.terrier.terms.BaseTermPipelineAccessor.<init>(BaseTermPipelineAccessor.java:70)
	at org.terrier.querying.ApplyTermPipeline.load_pipeline(ApplyTermPipeline.java:90

,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.448211,0.650289,0.592214,0.400000
1,XSqrA_M_costum,0.361886,0.640260,0.484063,0.298529
